In [30]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor



In [31]:
train_data = pd.read_csv(r'C:\Users\Benwari Ezekiel\Documents\code\house-prices-advanced-regression-techniques\train.csv')
#print(len(train_data.isnull().sum()))
numeric_cols = train_data.select_dtypes(include=['number']).columns
train_data[numeric_cols]= train_data[numeric_cols].fillna(train_data[numeric_cols].mean())
non_numeric_cols = train_data.select_dtypes(exclude=['number']).columns
#print(len(train_data[non_numeric_cols]))
train_data[non_numeric_cols]= train_data[non_numeric_cols].fillna('unknown')
print(train_data.describe())


                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1460.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    22.024023    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    60.000000    7553.500000     5.000000   
50%     730.500000    50.000000    70.049958    9478.500000     6.000000   
75%    1095.250000    70.000000    79.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  ...  \
count  1460.000000  1460.000000   1460.000000  1460.000000  1460.000000  ...   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726  ...   
std       1.112799    30.202904     20.645407   180.569112   456.098091  ..

In [32]:

le  = LabelEncoder()
#print(train_data)
test_data = pd.read_csv(r"C:\Users\Benwari Ezekiel\Documents\code\house-prices-advanced-regression-techniques\test.csv")
print(len(test_data.isnull().sum()))
#selct columns with numbers for test data set
numeric_cols2 = test_data.select_dtypes(include=['number']).columns
#fill the numeric data spaces missing with mean of the nueric data
test_data[numeric_cols2] = test_data[numeric_cols2].fillna(test_data[numeric_cols2].mean()) 
#select all non numeric data in test_data set
non_numeric_cols2 = test_data.select_dtypes(exclude=['number']).columns 
#fill with unknown 
test_data[non_numeric_cols2]= test_data[non_numeric_cols2].fillna('unknown')



80


In [33]:
print(train_data.shape,test_data.shape)
additional_row = pd.DataFrame(np.zeros((1,test_data.shape[1])),index=[test_data.shape[0]],columns=test_data.columns)
additional_cols = pd.DataFrame(np.zeros((test_data.shape[0],1)), index=range(test_data.shape[0]), columns = ['new_column'])
test_data=pd.concat([test_data,additional_row])
test_data= pd.concat([test_data,additional_cols],axis=1)

print(train_data.shape,test_data.shape)



(1460, 81) (1459, 80)
(1460, 81) (1460, 81)


In [34]:
print((test_data.isnull().sum()))
test_data.fillna(0)
train_data.fillna('unknown')


Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
new_column       1
Length: 81, dtype: int64


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,unknown,Reg,Lvl,AllPub,...,0,unknown,unknown,unknown,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,unknown,Reg,Lvl,AllPub,...,0,unknown,unknown,unknown,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,unknown,IR1,Lvl,AllPub,...,0,unknown,unknown,unknown,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,unknown,IR1,Lvl,AllPub,...,0,unknown,unknown,unknown,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,unknown,IR1,Lvl,AllPub,...,0,unknown,unknown,unknown,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,unknown,Reg,Lvl,AllPub,...,0,unknown,unknown,unknown,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,unknown,Reg,Lvl,AllPub,...,0,unknown,MnPrv,unknown,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,unknown,Reg,Lvl,AllPub,...,0,unknown,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,unknown,Reg,Lvl,AllPub,...,0,unknown,unknown,unknown,0,4,2010,WD,Normal,142125


In [36]:
#non numeric data colleted from test dat set as object whch is a collection of strings and numbers or just strings
non_numeric_cols = test_data.select_dtypes(include=['object']).columns
#converting each column index from a data set in to string then to numbers
for col in non_numeric_cols:
   test_data[col] = test_data[col].astype(str)
   test_data[col] = le.fit_transform(test_data[col])
non_numeric_cols  = train_data.select_dtypes(include= ['object']).columns
for cols in non_numeric_cols:
   train_data[cols]= train_data[cols].astype(str)
   train_data[cols]= le.fit_transform(test_data[cols])
   test_data2 = train_data['SalePrice']
   train_data2 = train_data.drop(columns=['SalePrice'])
   print(test_data2)
  
   x_train,x_test,y_train,y_test = train_test_split(train_data2,test_data2,test_size=0.2)
   


0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64
0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64
0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64
0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64
0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500

In [37]:
model = LinearRegression()
x_train = x_train.fillna(x_train.mean())
y_train=y_train.fillna(y_train.mean())
model.fit(x_train,y_train)
y_pred= model.predict(x_test)
mse = mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')






Mean Squared Error: 1100499479.713212
R² Score: 0.8267620035758444


In [38]:
param_grid ={
    'n_estimators':[50,100,200],
    'learning_rate':[0.01,0.1,0.2],
    'max_depth':[3,4,5]
}
gb_model = GradientBoostingRegressor()
grid_search = GridSearchCV(gb_model,param_grid,cv=5,scoring='neg_mean_squared_error')
#print(x_train.columns)
grid_search.fit(x_train, y_train)
print(grid_search.best_params_)



Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [41]:
model2 = GradientBoostingRegressor(learning_rate= 0.2, max_depth= 4, n_estimators= 200)
model2.fit(x_train,y_train)
y_pred = model2.predict(x_test)


In [44]:
r2_score_for_gb = r2_score(y_test,y_pred)
mse_score = mean_squared_error(y_test, y_pred)
print(r2_score_for_gb,mse_score)

0.8929834273428121 679825932.8835374
